In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

100%|██████████| 658M/658M [00:06<00:00, 114MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

data_dir = '/root/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1'

# Creating ImageDataGenerator with augmentation and validation split
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 33022 images belonging to 2 classes.
Found 8254 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load base model and freeze its layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Build new model on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
# Number of classes:
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2345s 2s/step - accuracy: 0.4981 - loss: 0.7198 - val_accuracy: 0.4989 - val_loss: 0.6932
Epoch 2/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2367s 2s/step - accuracy: 0.4971 - loss: 0.6935 - val_accuracy: 0.5001 - val_loss: 0.6931
Epoch 3/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2370s 2s/step - accuracy: 0.4996 - loss: 0.6933 - val_accuracy: 0.4993 - val_loss: 0.6932
Epoch 4/10
1032/1032 ━━━━━━━━━━━━━━━━━━━━ 2363s 2s/step - accuracy: 0.4999 - loss: 0.6933 - val_accuracy: 0.5002 - val_loss: 0.6932
Epoch 5/10
 218/1032 ━━━━━━━━━━━━━━━━━━━━ 24:27 2s/step - accuracy: 0.4999 - loss: 0.6931

In [ ]:
# Save model in SavedModel format
model.save('saved_model/plant_disease_detector')

# Convert model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('saved_model/plant_disease_detector')
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optional: enable optimizations
tflite_model = converter.convert()

# Save TFLite model
with open('plant_disease_detector.tflite', 'wb') as f:
    f.write(tflite_model)